# BUSINESS PROBLEM & UNDERSTANDING
STAKEHOLDER:SPOTIFY
PROBLEM: Spotify is launch a new Movie Divison. We are contracted Data Scientist tasked with  creating a reccomendation system for their movie division . We want to compete with other streaming platforms such as Netflix, Hulu, Amazon Prime. Since they are new to the market, they want to ensure that their Reccomendation System is accurate and reliable . This means creating a reliable reccomendation system that can provide users with movies and conetent taht tehy will actually enjoy thus keeping users on our platform.   

TASK:
Build a model that provides top 5 movie recommendations to a user, based on their ratings of other movies.
The MovieLens dataset is a "classic" recommendation system dataset, that is used in numerous academic papers and machine learning proofs-of-concept. You will need to create the specific details about how the user will provide their ratings of other movies, in addition to formulating a more specific business problem within the general context of "recommending movies".




# Data Preparation

In [ ]:
#Importing Libraries 
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt
import seaborn as sns
import surprise
from surprise.prediction_algorithms import *
import pandas as pd
import numpy as np
from datetime import datetime

from collections import Counter

from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split
%matplotlib inline

In [ ]:
# Reading in datasets
df_movies = pd.read_csv('data/movies.csv')
df_links = pd.read_csv('data/links.csv')
df_ratings = pd.read_csv('data/ratings.csv')
df_tags = pd.read_csv('data/tags.csv')

In [ ]:
#looking at each indivdual csv before merging.
df_movies.info()

In [ ]:
# This dataset contains links to each movie imdb and tmbd website ratings. Which is not applicable in the current scope of our business problem. 
df_links.info()


df_links dataset contains links to each movie imdb and tmbd website ratings. Which is not applicable in the current scope of our business problem. 

In [ ]:
df_ratings.info()


In [ ]:
df_tags.info()

In [ ]:
df_ratings

In [ ]:
# Checking for any movie duplicated in the dataset
df_movies[df_movies['title'].duplicated() == True]

Merging Dataset to one Pandas Database

In [ ]:
# combining the Movies, Ratings, Tags Dataset. Not including Links dataset due to its non relevance to the business problem.
movie_combo = pd.merge(df_movies, df_ratings, left_on='movieId', right_on='movieId', how='left')

movie_combo.info()

We can see there are missing values in rating and timestamps. Which will need to be addressed to ensure the algorithm will run sucessfully.

In [ ]:
movie_combo.head() # now we have combined all the relevant datsets into one pandas database.

In [ ]:
movie_combo.drop(['genres'], axis=1) 
# We want to drop genres due to its relevance in our collaborative filtering process.

Next we want to fill in the missing timestamp vales with the mode 

In [ ]:
# Filling in missing timestamp values with the mode. 
movie_combo['timestamp'] = movie_combo['timestamp'].fillna(movie_combo['timestamp'].mode()[0])
movie_combo.info()

In [ ]:
# Filling in missing rating values with the mode. 
movie_combo['rating'] = movie_combo['rating'].fillna(movie_combo['rating'].mode()[0])
movie_combo.info()

Next in our process  we want to convert the timestamp column to datetime format for our algorithm.
First we have to convert timestamp from float type to interger. 

In [ ]:
#converting timsestamp column to integers
movie_combo['timestamp'] = movie_combo['timestamp'].astype(int)

In [ ]:
movie_combo.head()

In [ ]:
#checking for assurance
movie_combo.info()

In [ ]:
movie_combo.head()

In [ ]:
# next is converting timestamp to datetime index format


movie_combo['timestamp'] = movie_combo['timestamp'].apply(lambda t: datetime.utcfromtimestamp(t*1).strftime('%Y-%m-%d'))
movie_combo.head()

In [ ]:
# Converting duplicate titles to the lower of 'movieId' duplicates as the lower value had more user ratings in each instance.
# This step was performed after merging the dateframes as we didn't want to lose data before the merge.

for idx, row in movie_combo.iterrows():
    # Emma (1996)
    if  movie_combo.loc[idx,'movieId'] == 26958:
        movie_combo.loc[idx,'movieId'] = 838
    # War of the Worlds (2005)
    elif movie_combo.loc[idx,'movieId'] == 64997:
        movie_combo.loc[idx,'movieId'] = 34048
    # Confessions of a Dangerous Mind (2002)
    elif movie_combo.loc[idx,'movieId'] == 144606:
        movie_combo.loc[idx,'movieId'] = 6003
    # Eros (2004)
    elif movie_combo.loc[idx,'movieId'] == 147002:
        movie_combo.loc[idx,'movieId'] = 32600
    # Saturn 3 (1980)
    elif movie_combo.loc[idx,'movieId'] == 168358:
        movie_combo.loc[idx,'movieId'] = 2851
    else:
        continue


In [ ]:
# Verify corrections (Should be blank)
movie_combo[(movie_combo['movieId'] == 26958 | 64997 | 144606 | 147002 | 168358)]

# Train Test Split

Next we begin with our train test split and get into our process of modeling

In [ ]:
#reader = Reader(rating_scale =(0.5, 5) ) 
#df = Dataset.load_from_df(new_df,reader)

In [ ]:

data = movie_combo[['userId', 'movieId', 'rating']]
reader = Reader(line_format='user item rating', sep=',',rating_scale=(0.5, 5))
data = Dataset.load_from_df(data,reader =reader)
# train-test-split
#trainset, testset = train_test_split(data, test_size=.2)


trainset, testset = surprise.model_selection.train_test_split(data, test_size=0.2, random_state=42)



# Baseline/ Dummy Model  KNN model

Baseline model with no paramater specified 

In [ ]:
knn_model = KNNBasic().fit(trainset)


In [ ]:
svd_model = SVD().fit(trainset)

In [ ]:
nmf_model = NMF().fit(trainset)

In [ ]:
from surprise import KNNWithMeans
from surprise.model_selection import GridSearchCV

sim_options = {
    "name": ["msd", "cosine","pearson"],
    "min_support": [3, 4],
    "user_based": [False, True]
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

In [ ]:
print(gs.best_score["mae"])
gs.best_params["mae"]

In [ ]:
param_grid = {
    "n_epochs": [5, 10],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.4, 0.6]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

In [ ]:
param_grid = {
    "n_neighbors": [3,4,5],
    "leaf_size": [3,4,5],
    "weights": ['uniform', 'distance'],
    "p": [1,2],
    "n_jobs": [-1]
}
gs = GridSearchCV(KNNBasic, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])
print(gs.best_score["mae"])
print(gs.best_params["mae"])

knn_model.sim

knn_model.test(testset)


In [ ]:
surprise.accuracy.mae(knn_model.test(testset))

In [ ]:
surprise.accuracy.rmse(knn_model.test(testset))

In [ ]:
surprise.accuracy.mae(svd_model.test(testset))

In [ ]:
surprise.accuracy.rmse(svd_model.test(testset))

In [ ]:
surprise.accuracy.mae(nmf_model.test(testset))

In [ ]:
surprise.accuracy.rmse(nmf_model.test(testset))

In [ ]:
knn_model_tuned = KNNBasic({'n_neighbors': 3, 'leaf_size': 5, 'n_jobs': -1}).fit(trainset)

In [ ]:
knn_model_tuned.test

In [ ]:
knn_model_tuned.test(testset)

In [ ]:
surprise.accuracy.mae(knn_model_tuned.test(testset))

In [ ]:
surprise.accuracy.rmse(knn_model_tuned.test(testset))

# Encoding 

One Hot Encoding since Genre is a Categorical Column. 

In [ ]:
#categorical_vars = ["genres"]
#one_hot_encoder = OneHotEncoder(sparse=False, drop = "first")

In [ ]:
#encoder_vars_array = one_hot_encoder.fit_transform(new_df[categorical_vars])
#encoder_feature_names = one_hot_encoder.get_feature_names(categorical_vars)
#encoder_vars_df = pd.DataFrame(encoder_vars_array, columns = encoder_feature_names)
#Data = pd.concat([new_df.reset_index(drop=True), encoder_vars_df.reset_index(drop=True)], axis = 1)
#Data.drop(categorical_vars, axis = 1, inplace = True)

# EDA and Visualizations 

In [ ]:
print('Number of users: ',len(Data.userId.unique()))
print('Number of movie titles: ',len(Data.movieId.unique()))
print('Number of ratings: ' ,Data.shape[0])
print('Number of tags: ' ,Data.tag.count())


In [ ]:
# groupby tags 
group = Data.groupby('tag').count()




group[:10].rating.plot(kind='bar', figsize=(15,6))



plt.title('Top 10 movie tags ');

In [ ]:
#top 10 re-occuring tags
top_tags =Data.groupby(by="tag").count().sort_values('movieId', ascending=False).userId.head(10)

top_tags


In [ ]:
# Distribution of ratings
Data.rating.plot.hist(bins=12)
plt.xlabel('Rating')
plt.title('Rating');